<a href="https://colab.research.google.com/github/N1a2t3a/.Data_Science/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import zipfile
import io
import os
import glob

In [48]:
zip_file_path = '/content/homework.zip'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/homework')

In [49]:
fixed_feature_count = 100

data_folder = 'homework/data/'

features = []
labels = []

actions = ['idle', 'running', 'stairs', 'walking']

for action in actions:
    action_folder = os.path.join(data_folder, action)

    for file_path in glob.glob(os.path.join(action_folder, '*.csv')):
        df = pd.read_csv(file_path).iloc[:, :fixed_feature_count]
        features.append(df)

        labels.append(action)

data = pd.concat(features, axis=0, ignore_index=True)

# Забезпечення відповідності розмірності
data = data.iloc[:len(labels)]

labels_df = pd.DataFrame(labels, columns=['label'])

X_train, X_test, y_train, y_test = train_test_split(data, labels_df, test_size=0.2, random_state=42)


In [50]:
common_length = 240
data = {
    'timestamp': pd.date_range(start='2022-01-01', periods=common_length, freq='1H'),
    'accelerometer_X': np.random.randn(common_length),
    'accelerometer_Y': np.random.randn(common_length),
    'accelerometer_Z': np.random.randn(common_length),
    'activity_label': np.random.choice(['idle', 'running', 'stairs', 'walking'], size=common_length)
}

df = pd.DataFrame(data)

print(df.head())

            timestamp  accelerometer_X  accelerometer_Y  accelerometer_Z  \
0 2022-01-01 00:00:00         1.514692         0.741007        -1.111742   
1 2022-01-01 01:00:00        -0.126940        -1.171602        -1.232466   
2 2022-01-01 02:00:00         0.630282        -1.806967        -0.655253   
3 2022-01-01 03:00:00         0.733423         0.122713         1.228252   
4 2022-01-01 04:00:00        -3.244493         1.342601         1.140174   

  activity_label  
0         stairs  
1           idle  
2        running  
3         stairs  
4           idle  


In [51]:
time_features = df.groupby('timestamp').agg(['mean', 'std', 'max', 'min'])

print("Processed Dataset Sample:\n", df.head())

Processed Dataset Sample:
             timestamp  accelerometer_X  accelerometer_Y  accelerometer_Z  \
0 2022-01-01 00:00:00         1.514692         0.741007        -1.111742   
1 2022-01-01 01:00:00        -0.126940        -1.171602        -1.232466   
2 2022-01-01 02:00:00         0.630282        -1.806967        -0.655253   
3 2022-01-01 03:00:00         0.733423         0.122713         1.228252   
4 2022-01-01 04:00:00        -3.244493         1.342601         1.140174   

  activity_label  
0         stairs  
1           idle  
2        running  
3         stairs  
4           idle  


<ipython-input-51-b5055cb96cd0>:1: FutureWarning: ['activity_label'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  time_features = df.groupby('timestamp').agg(['mean', 'std', 'max', 'min'])


In [55]:
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
X_train_scaled, X_test_scaled = X_train, X_test

In [56]:
svm_model = SVC()
svm_model.fit(X_train_scaled, y_train)

rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-56-ae2f747709b4>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train_scaled, y_train)


RandomForestClassifier()

In [57]:
svm_predictions = svm_model.predict(X_test_scaled)
print("SVM Classification Report:\n", classification_report(y_test, svm_predictions))

rf_predictions = rf_model.predict(X_test_scaled)
print("Random Forest Classification Report:\n", classification_report(y_test, rf_predictions))

SVM Classification Report:
               precision    recall  f1-score   support

        idle       0.00      0.00      0.00       220
     running       0.53      1.00      0.70       689
      stairs       0.00      0.00      0.00        30
     walking       0.00      0.00      0.00       354

    accuracy                           0.53      1293
   macro avg       0.13      0.25      0.17      1293
weighted avg       0.28      0.53      0.37      1293

Random Forest Classification Report:
               precision    recall  f1-score   support

        idle       0.17      0.10      0.13       220
     running       0.57      0.67      0.61       689
      stairs       0.00      0.00      0.00        30
     walking       0.29      0.28      0.28       354

    accuracy                           0.45      1293
   macro avg       0.26      0.26      0.26      1293
weighted avg       0.41      0.45      0.43      1293



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
